<a href="https://colab.research.google.com/github/Chrissek/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

from hyperopt import hp, fmin, tpe, STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car" 

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [13]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')))



In [0]:
def run_model(model, feats):
  x = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, x, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [20]:
feats = ['param_napęd__cat',
'param_rok-produkcji',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc',
'param_stan__cat',
'feature_kamera-cofania__cat',
'param_pojemność-skokowa',
'feature_łopatki-zmiany-biegów__cat',
'feature_światła-led__cat',
'feature_regulowane-zawieszenie__cat',
'seller_type__cat',
'feature_wspomaganie-kierownicy__cat',
'seller_name__cat',
'feature_elektrycznie-ustawiane-fotele__cat',
'param_wersja__cat',
'feature_system-start-stop__cat',
'feature_bluetooth__cat',
'param_typ__cat',
'feature_czujniki-parkowania-przednie__cat']
xgb_params = {
    'max_depth':5,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'seed': 0
}
run_model(xgb.XGBRegressor(**xgb_params), feats)

[21:01:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:01:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:01:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-10003.559859789288, 71.63849140290796)

## Hyperopt

In [27]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**xgb_params), feats)

  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}


# space
xgb_reg_params = {
    'leraning_rate':  hp.choice('learning_rate',  np.arange(0.05, 0.31, 0.05)),
    'max_depth':      hp.choice('max_depth',      np.arange(5, 16, 1, dtype=int)),
    'subsample':      hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree':hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':      'reg:squarederror',
    'n_estimators':   100,
    'seed': 0,
}


#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best


Training with params: 
{'colsample_bytree': 0.8, 'leraning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
[21:46:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:46:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:46:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Training with params: 
{'colsample_bytree': 0.55, 'leraning_rate': 0.1, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
[21:47:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:47:11

{'colsample_bytree': 0.8, 'learning_rate': 1, 'max_depth': 1, 'subsample': 0.8}